# Working with acoustic communication data in VocalPy

This tutorial introduces us to VocalPy, that we'll use in the other notebooks.

It is adapted from https://vocalpy.readthedocs.io/en/latest/getting_started/quickstart.html

### Let's start with some example data

In [ ]:
import vocalpy as voc
import librosa

VocalPy has built-in examples of acoustic communication data.

In [ ]:
voc.examples.show()

If we just give it the name of an example, we get back a path to a file.

In [ ]:
example = voc.example('samba.wav')
print(example)

## Data types for acoustic communication

We'll use the example data to look at the data types that VocalPy provides for acoustic comunication.

### Data type for sound: `vocalpy.Sound`

We read a sound in from a file with the `Sound.read` method

In [ ]:
sound = voc.Sound.read(example)

We now have a data container with the sound `data` itself as well as the `samplerate` as an attribute.

In [ ]:
sound

In [ ]:
sound.data

In [ ]:
sound.samplerate

A `Sound` also has three properties, derived from its data:

1. channels, the number of channels

2. samples, the number of samples, and

3. duration, the number of samples divided by the sampling rate.


In [ ]:
print(
    f"This sound comes from an audio file with {sound.channels} channel, "
    f"{sound.samples} samples, and a duration of {sound.duration:.3f} seconds"
)

One of the reasons VocalPy provides this data type, and the others we’re about to show you here, is that it helps you write more succinct code that’s easier to read: for you, when you come back to your code months from now, and for others that want to read the code you share.

The properties and attributes let us avoid writing a bunch of variable names like `data` and `fs` and `samplerate` that clutter up our code, when what we really want to think about are the sound itself.

Let's visually inspect the sound. Notice that if we need the data as a NumPy array, we can use the `data` attribute. This lets VocalPy work well with other Python packages, such as `librosa` (https://librosa.org/).

In [ ]:
librosa.display.waveshow(sound.data)

We can see some peaks suggesting activity, but we can't tell much else about this sound just by looking at it as a time series.

## Data type: `vocalpy.Spectrogram`

* Intuitively: a picture of sound
* Technically speaking: "the squared magnitude of the Short Time Fourier Transform"

In [ ]:
spect = voc.spectrogram(sound)

As before, we’ll walk through the attributes of this class. But since the whole point of a spectrogram is to let us see sound, let’s actually look at the spectrogram, instead of staring at arrays of numbers.  

We do so by calling `vocalpy.plot.spectrogram`.

In [ ]:
voc.plot.spectrogram(spect)

Now we can see that we are working with zebra finch song!  

Now that we know what we’re working with, let’s actually inspect the attributes of the vocalpy.Spectrogram instance.

In [ ]:
spect

There are five attributes we care about here.

1. `data`: this is the spectrogram itself – as with the other data types, like `vocalpy.Sound`, the attribute name `data` indiciates this main data we care about

In [ ]:
spect.data

Let’s look at the shape of `data`. It’s really just a NumPy array, so we inspect the array’s shape attribute.

In [ ]:
spect.data.shape

We see that we have an array with dimensions (channels, frequencies, times). The last two dimensions correspond to the next two attributes we will look at.

2. `frequencies`, a vector of the frequency for each row of the spectrogram.

In [ ]:
print(spect.frequencies[:10])

In [ ]:
print(spect.frequencies.shape)

(We see it is equal to the number of elements in the second dimension of `data`.)

3. `times`, a vector of the time for each column in the spectrogram.

In [ ]:
spect.times

Just like with the `Sound` class, VocalPy gives us the ability to conveniently read and write spectrograms from files. This saves us from generating spectrograms over and over. Computing spectrograms can be computionally expensive, if your audio has a high sampling rate or you are using methods like multi-taper spectrograms. Saving spectrograms from files also makes it easier for you to share your data in the exact form you used it, so that it’s easier to replicate your analyses.

In [ ]:
import pathlib

DATA_DIR = pathlib.Path('./data/my-data')
DATA_DIR.mkdir(exist_ok=True)

spect_file = spect.write(DATA_DIR / f"{spect.audio_path.name}.spect.npz")

Notice that the extension is `'npz'`; this is a file format that NumPy uses to save mulitple arrays in a single file: We are saving the `data`, the `frequencies` and the `times`.

By convention we include the file extension of the source audio, and another “extension” that incidicates this is a spectrogram, so that the file name ends with `'.wav.spect.npz'`.

## Classes for steps in pipelines for processing data in acoustic communication

In addition to data types for acoustic communication, VocalPy provides you with classes that represent steps in pipelines for processing that data. These classes are also written with readability and reproducibility in mind.

Let's say we want to make spectrograms for all of our sound data. We'll use one of the classes, `SpectrogramMaker`, to make a spectrogram from each one of a set of wav files.

When you are working with your own data, instead of example data built into VocalPy, you will do something like:

1. Load all the sound files from a directory using a convenience function that VocalPy gives us in its paths module, `vocalpy.paths.from_dir`

2. Load all the wav files into the data type that VocalPy provides for sound, `vocalpy.Sound`, using the method `vocalpy.Sound.read`

This is shown in the snippet below.

In [ ]:
bfsongrepo_dir = './data/Nicholson-Queen-Sober-2017-bfsongrepo-subset/'
wav_paths = voc.paths.from_dir(dir=bfsongrepo_dir, ext='wav')

Read in all the `Sound`s with a list comprehension

In [ ]:
sounds = [
    voc.Sound.read(wav_path)
    for wav_path in wav_paths
]

Now we use the `SpectrogramMaker` class to make spectrograms from all these `Sound`s.

To make sure our code is readable to "future us", we write down the function we used as a `callback`, and we write down the parameters.

In [ ]:
# this is the function we used
callback = voc.spectrogram
# these are the parameters we used
params = dict(n_fft=512, hop_length=64)
spect_maker = voc.SpectrogramMaker(
    callback, params
)

In [ ]:
print(spect_maker.params)

In [ ]:
spects = spect_maker.make(sounds, parallelize=True)

Let's plot four of the spectrograms we made.

In [ ]:
import matplotlib.pyplot as plt
fig, ax_arr = plt.subplots(2, 2, figsize=(10, 5))
ax_arr = ax_arr.ravel()

for ax, spect in zip(ax_arr, spects):
    voc.plot.spectrogram(spect, ax=ax)

## Data type: `vocalpy.Annotation`

The last data type we’ll look at is for annotations. Such annotations are important for analysis of aocustic communication and behavior. Under the hood, VocalPy uses the pyOpenSci package `crowsetta`(https://github.com/vocalpy/crowsetta).

In [ ]:
csv_paths = voc.paths.from_dir('data/Nicholson-Queen-Sober-2017-bfsongrepo-subset/', 'csv')

In [ ]:
csv_paths[:5]

In [ ]:
annots = [
    voc.Annotation.read(csv_path, format='simple-seq')
    for csv_path in csv_paths
]

We inspect one of the annotations. Again as with other data types, we can see there is a `data` attribute. In this case it contains the `crowsetta.Annotation`.

In [ ]:
print(annots[0])

We plot the spectrogram along with the annotations.

In [ ]:
voc.plot.annotated_spectrogram(spects[0], annots[0], tlim=[1.8, 4]);

This crash course in VocalPy has introduced you to the key features and goals of the library. To learn more, please check out the [documentation](https://vocalpy.readthedocs.io/en/latest/), read our Forum Acusticum 2023 Proceedings Paper, [“Introducing VocalPy”](https://dael.euracoustics.org/confs/fa2023/data/articles/000403.pdf), and watch the talks ["VocalPy: a core Python package for acoustic communication research"](https://www.youtube.com/watch?v=53S5xM6s70g) and ["VocalPy as a cast study of domain-driven design in scientific Python"](https://www.youtube.com/watch?v=PtTegIM6m1o). We are actively developing the library to meet your needs and would love to hear your feedback in [our forum](https://forum.vocalpy.org/).